<a href="https://colab.research.google.com/github/AshleynLai/Nyc_airbn_listings/blob/main/nyc_listing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import io
import re
re.compile('<title>(.*)</title>')

Saving nyc_listing.csv to nyc_listing.csv


CHECK DATASET

In [3]:

for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)
df = pd.read_csv(io.BytesIO(uploaded['nyc_listing.csv']))
print(df.head(5))

saved file nyc_listing.csv
     id                                               name  host_id  \
0  2595  Rental unit in New York · ★4.68 · Studio · 1 b...     2845   
1  5121  Rental unit in Brooklyn · ★4.52 · 1 bedroom · ...     7356   
2  6848  Rental unit in Brooklyn · ★4.58 · 2 bedrooms ·...    15991   
3  6872  Condo in New York · 1 bedroom · 1 bed · 1 shar...    16104   
4  7064  Loft in Brooklyn · ★4.91 · 1 bedroom · 1 bed ·...    17297   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0       Jennifer           Manhattan             Midtown  40.75356  -73.98559   
1          Garon            Brooklyn  Bedford-Stuyvesant  40.68535  -73.95512   
2  Allen & Irina            Brooklyn        Williamsburg  40.70935  -73.95342   
3            Kae           Manhattan         East Harlem  40.80107  -73.94255   
4         Joelle            Brooklyn        Williamsburg  40.71248  -73.95881   

         room_type  price  minimum_nights  number_of_review

EXTRACT DATA FROM COLUMN 'NAME'

In [4]:
# Define a function to extract the relevant information
def extract_info(row):
    try:
        segments = row['name'].split(' · ')
        PropertyType = segments[0]
        Rating = None
        Bedrooms = None
        Beds = None
        Baths = None

        for segment in segments:
            if '★' in segment:
                Rating = segment.replace('★', '')
            elif 'bedroom' in segment or 'bedrooms' in segment:
                Bedrooms = segment.split()[0]
            elif 'Studio' in segment:
                Bedrooms = 1
            elif 'bed' in segment or 'beds' in segment:
                Beds = segment.split()[0]
            elif 'bath' in segment or 'baths' in segment or 'Half-bath' in segment:
                Baths = segment.split()[0]

        return pd.Series([PropertyType, Rating, Bedrooms, Beds, Baths])
    except Exception as e:
        # Handle unexpected data or errors
        return pd.Series([None, None, None, None, None])

# Apply the function to split and extract information
df[['PropertyType', 'Rating', 'Bedrooms', 'Beds', 'Baths']] = df.apply(extract_info, axis=1)
df['PropertyType'] = df['PropertyType'].str.split(' in ').str[0]

# Display the updated DataFrame
print(df)

                       id                                               name  \
0                    2595  Rental unit in New York · ★4.68 · Studio · 1 b...   
1                    5121  Rental unit in Brooklyn · ★4.52 · 1 bedroom · ...   
2                    6848  Rental unit in Brooklyn · ★4.58 · 2 bedrooms ·...   
3                    6872  Condo in New York · 1 bedroom · 1 bed · 1 shar...   
4                    7064  Loft in Brooklyn · ★4.91 · 1 bedroom · 1 bed ·...   
...                   ...                                                ...   
38787  992059954042123977  Rental unit in Brooklyn · ★New · 1 bedroom · 1...   
38788  992091077050127349  Rental unit in New York · ★New · 1 bedroom · 1...   
38789  992096613198031825  Rental unit in New York · ★New · 1 bedroom · 1...   
38790  992729500548386711  Rental unit in New York · ★New · 3 bedrooms · ...   
38791  992729511234270784  Rental unit in New York · ★New · 1 bedroom · 1...   

         host_id      host_name neighbo

ELIMINATE UNNECESSARY COLUMNS AND FORMAT COLUMNS TO MEET MYSQL STANDARDS

In [5]:
df = df.drop(columns=['name', 'latitude', 'longitude'])
df['host_name'] = df['host_name'].str.replace(',', ' and')
df['neighborhood'] = df['room_type'].str.replace(',', ' and')
df['id'] = df['id'].astype(str).str[:8]
print(df.head(50))

       id  host_id       host_name neighbourhood_group       neighbourhood  \
0    2595     2845        Jennifer           Manhattan             Midtown   
1    5121     7356           Garon            Brooklyn  Bedford-Stuyvesant   
2    6848    15991   Allen & Irina            Brooklyn        Williamsburg   
3    6872    16104             Kae           Manhattan         East Harlem   
4    7064    17297          Joelle            Brooklyn        Williamsburg   
5    7097    17571            Jane            Brooklyn         Fort Greene   
6    7801    21207           Chaya            Brooklyn        Williamsburg   
7    8490    25183        Nathalie            Brooklyn  Bedford-Stuyvesant   
8    9357    30193           Tommi           Manhattan      Hell's Kitchen   
9   10452    35935          Angela            Brooklyn  Bedford-Stuyvesant   
10  11943    45445         Harriet            Brooklyn            Flatbush   
11  12192    46978          Edward           Manhattan        Ea

DOWNLOAD CLEANED DATASET

In [6]:
from google.colab import files
df.to_csv('nyc_listings1.csv')
files.download('nyc_listings1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>